In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import tqdm 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import random
import os


In [2]:
train_labels0=[]
test_labels0=[]
train_imgs0=[]
test_imgs0=[]
classes=['Bear', 'Bull', 'Cat', 'Cattle',
           'Cheetah', 'Chicken', 'Crocodile',
           'Deer', 'Dog', 'Fox', 'Goat', 'Hippopotamus', 
           'Horse', 'Jaguar', 'Leopard', 'Lion', 'Lynx',
           'Monkey', 'Mule', 'Ostrich', 'Owl', 
           'Pig', 'Raccoon', 'Raven', 'Rhinoceros', 
           'Sheep', 'Snake', 'Tiger', 'Turkey', 'Turtle', 'Zebra']
"""classes=['Cat','Dog']"""
encode={}
for i in range(len(classes)):
    encode[classes[i]]=i+1

def checkclass(dirname):
    
    for animal in classes:
        if dirname.endswith(animal):
            return True
    return False

for dirname, dirs, filenames in os.walk('./data/'):
    
    if 'train' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        
        for label in sorted(os.listdir(labels)):
            
            
            if label.endswith('txt'):
                train_labels0.append(labels+label)
        
        for image in sorted(filenames):
            
            
            if image.endswith('png') or image.endswith('jpg'):
                train_imgs0.append(dirname+'/'+image)
    if 'val' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        for label in sorted(os.listdir(labels)):
            if label.endswith('.txt'):
                test_labels0.append(labels+label)
        for image in sorted(filenames):
            if image.endswith('png')or image.endswith('jpg'):
                test_imgs0.append(dirname+'/'+image)


In [3]:
print(len(train_imgs0))
print(len(train_labels0))
print(len(test_imgs0))
print(len(test_labels0))
train_labels=[]
test_labels=[]
train_imgs=[]
test_imgs=[]
for i in range(len(train_labels0)):
   
        
        train_labels.append(train_labels0[i])
        train_imgs.append(train_imgs0[i])
for i in range(len(test_labels0)):
   
        test_labels.append(test_labels0[i])
        test_imgs.append(test_imgs0[i])
print(len(train_imgs))
print(len(train_labels))
print(len(test_imgs))
print(len(test_labels))


14620
14620
6864
6864
14620
14620
6864
6864


In [4]:
class AnimalDataset(object):
    def __init__(self, transforms,img_dir,label_dir):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = img_dir
        self.label=label_dir
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = self.imgs[idx]
        file_label = self.label[idx]
        img = Image.open(file_image).convert("RGB")
        target={}
        target['image_id']=torch.tensor([idx])
        labels=[]
        boxes=[]
        #Generate Label
        with open(file_label,'r')as f:
            lines=f.readlines()
            for line in lines:
                line_list=line.strip().split(' ')
                labels.append(encode[line_list[0]])
                boxes.append([float(i) for i in line_list[1:]])
                
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)        
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        iscrowd = torch.zeros((len(classes),), dtype=torch.int64)

        target['boxes']=boxes
        target['labels']=labels
       
        target["area"] = area
        target["iscrowd"] = iscrowd
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [5]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [6]:
import torchvision
from torchvision.models.detection.ssd import SSD
from torchvision.models.detection.rpn import AnchorGenerator
def get_model_instance_mobilenet(num_classes):
    backbone = torchvision.models.mobilenet_v3_small(weights="DEFAULT").features
    
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                       aspect_ratios=((0.5, 1.0, 2.0),))
    size=(300,300)
    model=SSD(backbone=backbone,anchor_generator=anchor_generator,size=size,num_classes=num_classes)
    return model

In [7]:
from engine import train_one_epoch, evaluate
import utils
import time
train_data=AnimalDataset(get_transform(train=False),train_imgs,train_labels)
val_data=AnimalDataset(get_transform(train=False),train_imgs[:100],train_labels[:100])
test_data=AnimalDataset(get_transform(train=False),test_imgs[:100],test_labels[:100])
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person


# define training and validation data loaders
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=4, shuffle=True, num_workers=0,
    collate_fn=utils.collate_fn)
val_loader = torch.utils.data.DataLoader(
    val_data, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)

# get the model using our helper function

model =get_model_instance_mobilenet(len(classes)+1)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=10,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 50
starttime=time.time()

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=100)
    # update the learning rate
    torch.save(model.state_dict(), "./model/SSD/SSD_mobilenet"+str(epoch)+".pt")
    
    print('saved dict')
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model,val_loader, device=device)
    evaluate(model,test_loader, device=device)
print("training time: "+str(time.time()-starttime)+"s")

KeyboardInterrupt: 

In [12]:
import time
from PIL import Image, ImageDraw,ImageFont
i=6000
model=get_model_instance_mobilenet(len(classes)+1)
model.load_state_dict(torch.load('./model/SSD/SSD_mobilenet19.pt'))
model.to('cuda')
model.eval()
test_data=AnimalDataset(get_transform(train=False),test_imgs[i:i+1],test_labels[i:i+1])
# Load a batch of images
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

timecount=0
for imgs,target in test_loader:
    images = list(img.to('cuda') for img in imgs)
    torch.cuda.synchronize()
    start_time = time.time()
    # Run the model on the images
    with torch.no_grad():
        
        outputs = model(images)
    timecount+=time.time()-start_time
    # Measure the end time
    print(outputs)
    max_pred=torch.argmax(outputs[0]['scores'])
    boxes = [outputs[0]['boxes'][max_pred]   ]  
    # Load the image using PIL
    

    # Draw the boxes on the image
    
        

    # Display the image
    img = Image.open(test_imgs[i])

    # Create a drawing object
    draw = ImageDraw.Draw(img)

    # Define the font to use for the labels
    font = ImageFont.truetype("/usr/share/fonts/truetype/abyssinica/AbyssinicaSIL-Regular.ttf", 16)

    # Define the bounding boxes and labels
    
    labels = [classes[outputs[0]['labels'][max_pred]-1]]
    print(labels)
    print(boxes)
    # Draw the bounding boxes and labels
    for box, label in zip(boxes, labels):
        draw.rectangle(box.tolist(), outline="red", width=2)
        draw.text((box), label, font=font)

    # Display the image
    img.show()
# Measure the start time



[{'boxes': tensor([[ 195.3854,   92.4113,  895.0213,  573.6733],
        [ 141.4835,   82.9564,  877.0256,  562.1683],
        [ 141.4835,   82.9564,  877.0256,  562.1683],
        [ 155.3709,  124.3830,  662.4296,  448.5272],
        [ 865.8257,  471.3360, 1000.8845,  653.0099],
        [ 247.0153,   79.3818,  916.9377,  669.1871],
        [ 144.6983,   86.3108,  860.6525,  610.2250],
        [ 247.0153,   79.3818,  916.9377,  669.1871],
        [ 869.1481,  428.2607, 1024.0000,  658.7056],
        [ 766.2272,  496.4095,  837.7216,  574.1753],
        [ 912.6890,  527.6732, 1019.4130,  669.8196]], device='cuda:0'), 'scores': tensor([0.9884, 0.0915, 0.0738, 0.0296, 0.0208, 0.0187, 0.0176, 0.0124, 0.0103,
        0.0102, 0.0101], device='cuda:0'), 'labels': tensor([ 6, 12, 29,  6,  3, 27, 30, 18,  9, 13, 22], device='cuda:0')}]
['Chicken']
[tensor([195.3854,  92.4113, 895.0213, 573.6733], device='cuda:0')]
